## Section 1: Setup & Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Paths
BASE = Path.cwd()
MERGED_PATH = BASE / 'DATA_CLEANED' / 'processed' / 'merged_dataset.parquet'
OUTPUT_DIR = BASE / 'DATA_CLEANED' / 'processed'

# Load data
print("Loading merged dataset...")
df = pd.read_parquet(MERGED_PATH)
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns[:10])}... (first 10)")

## Section 2: Remove Target Variable

⚠️ **CRITICAL**: Drop the fire column - we don't want to guide clustering

In [ ]:
print("="*80)
print("CLUSTERING FEATURE SELECTION (UNSUPERVISED)")
print("="*80)

# Drop target variable
df_features = df.drop('fire', axis=1).copy()

print(f"\n1. Removed target variable 'fire'")
print(f"   Remaining features: {df_features.shape[1]}")

# Also remove coordinates (they're not environmental features for clustering)
coords_to_drop = [c for c in df_features.columns if c in ['latitude', 'longitude']]
if coords_to_drop:
    print(f"\n2. Removed spatial coordinates: {coords_to_drop}")
    df_features = df_features.drop(coords_to_drop, axis=1)
    print(f"   Remaining features: {df_features.shape[1]}")

print(f"\n✓ Data prepared for clustering analysis")

## Section 3: Variance Analysis

Remove features with near-zero variance

In [ ]:
print("\n" + "="*80)
print("VARIANCE ANALYSIS")
print("="*80)

# Calculate variance
variances = df_features.var()
std_devs = df_features.std()

print(f"\nFeature Variance Statistics:")
print(f"  Mean variance: {variances.mean():.4f}")
print(f"  Median variance: {variances.median():.4f}")
print(f"  Min variance: {variances.min():.6f}")
print(f"  Max variance: {variances.max():.4f}")

# Find near-zero variance features
variance_threshold = variances.mean() * 0.01  # Features with <1% of mean variance
low_var_features = variances[variances < variance_threshold].index.tolist()

if low_var_features:
    print(f"\nFeatures with near-zero variance (< {variance_threshold:.6f}):")
    for feat in low_var_features:
        print(f"  - {feat}: variance={variances[feat]:.6f}, std={std_devs[feat]:.6f}")
    
    print(f"\nRemoving {len(low_var_features)} low-variance features...")
    df_features = df_features.drop(low_var_features, axis=1)
else:
    print(f"\n✓ No near-zero variance features found")

print(f"\nRemaining features: {df_features.shape[1]}")

## Section 4: Correlation Analysis

Remove highly correlated features (keep one from each correlated pair)

## Section 4: Correlation Analysis

Remove highly correlated features (keep one from each correlated pair)

In [ ]:
print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

# Calculate correlation matrix
corr_matrix = df_features.corr().abs()

# Find pairs with high correlation
correlation_threshold = 0.9
print(f"\nFeatures with correlation > {correlation_threshold}:")

high_corr_pairs = []
upper_triangle = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
for i in np.where(upper_triangle):
    if corr_matrix.iloc[i[0], i[1]] > correlation_threshold:
        feat1 = corr_matrix.index[i[0]]
        feat2 = corr_matrix.columns[i[1]]
        corr_val = corr_matrix.iloc[i[0], i[1]]
        high_corr_pairs.append((feat1, feat2, corr_val))
        print(f"  {feat1} <-> {feat2}: {corr_val:.3f}")

# Strategy: Keep feature with higher mean correlation to other features
features_to_drop = set()
for feat1, feat2, corr in high_corr_pairs:
    mean_corr_1 = corr_matrix[feat1].mean()
    mean_corr_2 = corr_matrix[feat2].mean()
    
    if mean_corr_1 > mean_corr_2:
        features_to_drop.add(feat2)
    else:
        features_to_drop.add(feat1)

if features_to_drop:
    print(f"\nRemoving {len(features_to_drop)} redundant features to reduce multicollinearity...")
    df_features = df_features.drop(list(features_to_drop), axis=1)
    print(f"Removed: {list(features_to_drop)}")
else:
    print(f"\n✓ No highly correlated features found")

print(f"\nRemaining features: {df_features.shape[1]}")

# Visualize remaining correlation matrix
plt.figure(figsize=(12, 10))
new_corr = df_features.corr()
sns.heatmap(new_corr, annot=False, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={'label': 'Correlation'})
plt.title(f'Feature Correlation Matrix (After Filtering)\nn={df_features.shape[1]} features', 
          fontweight='bold', fontsize=12)
plt.tight_layout()
plt.show()

## Section 5: Domain-Based Feature Selection

Select features relevant to fire ecology and environmental analysis

In [ ]:
print("\n" + "="*80)
print("DOMAIN-BASED FEATURE SELECTION")
print("="*80)

# Categorize features by domain
feature_categories = {
    'Climate': [c for c in df_features.columns if any(x in c.lower() for x in ['temp', 'prec', 'tmax', 'tmin'])],
    'Topography': [c for c in df_features.columns if any(x in c.lower() for x in ['elev', 'slope', 'aspect', 'rough'])],
    'Soil': [c for c in df_features.columns if any(x in c.upper() for x in ['SAND', 'CLAY', 'SILT', 'CARBON', 'PH', 'CEC', 'BULK'])],
    'Landcover': [c for c in df_features.columns if 'lulc' in c.lower() or 'landcover' in c.lower()],
    'Engineered': [c for c in df_features.columns if any(x in c.lower() for x in ['index', 'ratio', 'range', 'variability', 'interaction'])]
}

# Print categories
print(f"\nFeature Categories:")
for category, features in feature_categories.items():
    if features:
        print(f"\n  {category} ({len(features)} features):")
        for feat in features[:5]:  # Show first 5
            print(f"    - {feat}")
        if len(features) > 5:
            print(f"    ... and {len(features)-5} more")

# Select best features from each category
print(f"\n\nSelecting features from each category...")

selected_features = []

# Climate: Select most important ones
climate_features = feature_categories['Climate']
if climate_features:
    # Prefer seasonal aggregates over raw monthly
    selected_climate = [c for c in climate_features if any(x in c.lower() for x in ['summer', 'winter', 'total', 'avg'])]
    selected_features.extend(selected_climate[:4])  # Max 4 climate features
    print(f"  ✓ Climate: {len(selected_climate[:4])} features")

# Topography: Most important for fire spread
topo_features = feature_categories['Topography']
if topo_features:
    selected_features.extend(topo_features[:4])  # Max 4 topo features (usually: elev, slope, aspect)
    print(f"  ✓ Topography: {len(topo_features[:4])} features")

# Soil: Select diverse properties
soil_features = feature_categories['Soil']
if soil_features:
    # Prefer variety: texture, chemistry, fertility
    soil_texture = [c for c in soil_features if any(x in c for x in ['SAND', 'CLAY', 'SILT'])]
    soil_chem = [c for c in soil_features if any(x in c for x in ['PH', 'CARBON', 'WATER'])]
    soil_fertility = [c for c in soil_features if any(x in c for x in ['CEC', 'BULK', 'N'])]
    
    selected_soil = soil_texture[:2] + soil_chem[:2] + soil_fertility[:2]
    selected_features.extend(selected_soil)
    print(f"  ✓ Soil: {len(selected_soil)} features")

# Landcover: All if available
lc_features = feature_categories['Landcover']
if lc_features:
    selected_features.extend(lc_features)
    print(f"  ✓ Landcover: {len(lc_features)} features")

# Engineered: Add if helpful
eng_features = feature_categories['Engineered']
if eng_features:
    selected_features.extend(eng_features[:3])  # Max 3 engineered
    print(f"  ✓ Engineered: {len(eng_features[:3])} features")

# Final selection
selected_features = list(set(selected_features))  # Remove duplicates
if len(selected_features) > 20:
    # If too many, select top 20 by variance
    selected_features = df_features[selected_features].var().nlargest(20).index.tolist()

print(f"\n\nFinal Selected Features ({len(selected_features)} total):")
for i, feat in enumerate(sorted(selected_features), 1):
    print(f"  {i:2d}. {feat}")

## Section 6: Scale and Prepare Final Dataset

## Section 6: Scale and Prepare Final Dataset

In [ ]:
print("\n" + "="*80)
print("DATA PREPARATION FOR CLUSTERING")
print("="*80)

# Select final features
X_clustering = df_features[selected_features].copy()

print(f"\nDataset shape: {X_clustering.shape}")
print(f"Features: {X_clustering.shape[1]}")
print(f"Samples: {X_clustering.shape[0]}")

# Check for missing values
missing = X_clustering.isnull().sum().sum()
if missing > 0:
    print(f"\nFound {missing} missing values, imputing with median...")
    X_clustering = X_clustering.fillna(X_clustering.median())
else:
    print(f"\n✓ No missing values")

# Standardization for clustering (important for distance-based algorithms)
print(f"\nStandardizing features (zero-mean, unit-variance)...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clustering)
X_scaled_df = pd.DataFrame(X_scaled, columns=selected_features)

print(f"\nScaled data statistics:")
print(f"  Mean: {X_scaled.mean(axis=0).mean():.6f} (should be ~0)")
print(f"  Std: {X_scaled.std(axis=0).mean():.6f} (should be ~1)")
print(f"\n✓ Data preparation complete!")

## Section 7: Save Clustering Dataset

In [ ]:
print("\n" + "="*80)
print("SAVING CLUSTERING DATASET")
print("="*80)

import json

# Save standardized data
output_path = OUTPUT_DIR / 'dataset_for_clustering.parquet'
X_scaled_df.to_parquet(output_path)
print(f"\n✓ Saved standardized clustering dataset:")
print(f"  Path: {output_path}")
print(f"  Shape: {X_scaled_df.shape}")

# Save feature metadata
feature_metadata = {
    'selected_features': selected_features,
    'n_features': len(selected_features),
    'feature_categories': {
        category: [f for f in features if f in selected_features]
        for category, features in feature_categories.items()
    },
    'preprocessing': {
        'removed_low_variance': low_var_features if 'low_var_features' in locals() else [],
        'removed_correlated': list(features_to_drop) if 'features_to_drop' in locals() else [],
        'standardization': 'StandardScaler (mean=0, std=1)',
        'removed_target': True,
        'removed_coordinates': True
    }
}

metadata_path = OUTPUT_DIR / 'clustering_feature_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(feature_metadata, f, indent=2)
print(f"\n✓ Saved feature metadata:")
print(f"  Path: {metadata_path}")

# Save scaler for later use
import pickle
scaler_path = OUTPUT_DIR / 'clustering_scaler.pkl'
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"\n✓ Saved StandardScaler:")
print(f"  Path: {scaler_path}")

print(f"\n" + "="*80)
print("CLUSTERING DATASET PREPARATION COMPLETE!")
print("="*80)
print(f"\n📊 Summary:")
print(f"  ✓ Selected {len(selected_features)} features (out of {df_features.shape[1]})")
print(f"  ✓ Removed low-variance features")
print(f"  ✓ Removed highly correlated features")
print(f"  ✓ Removed target variable (unsupervised)")
print(f"  ✓ Standardized all features")
print(f"\n✅ Ready for: K-Means, DBSCAN, CLARANS clustering")